<a href="https://colab.research.google.com/github/bisil2/AI1_Final_Project/blob/main/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A51_%EB%AA%A8%EB%8D%B8_%ED%95%99%EC%8A%B5(Inception_v4_%EC%82%AC%EC%A0%84%ED%95%99%EC%8A%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고자료

https://deep-learning-study.tistory.com/525
https://rahites.tistory.com/177

https://huggingface.co/timm/inception_v4.tf_in1k

In [ ]:
use_background = 1
use_agumentation = 1
use_dropout = 1
use_l2 = 1

## 라이브러리 설치 및 Import

In [ ]:
''' 패키지 설치 '''
!pip install torch torchvision

# =============================
# torch : 모델 실행, 학습, 추론에 필수적인 PyTorch 프레임워크
# torchvision : 이미지 처리 관련 도구 제공
# =============================

In [ ]:
'''Google Drive 연동'''
from google.colab import drive
drive.mount('/content/drive')

'''필수 라이브러리 import'''
import pandas as pd
import numpy as np
import shutil
import zipfile
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 불러오기

In [ ]:
'''Label 데이터 불러오기'''
rawdata = pd.read_csv("/content/drive/MyDrive/data/rawdata.csv")

In [ ]:
'''Image 데이터 불러오기'''
if use_background:
  zipName = 'data'
else:
  zipName = 'raw'

# data 폴더 생성
targetPath = '/content/data/'
os.makedirs(targetPath, exist_ok=True)

# 압축 파일 content로 복사 => content에 있으면 처리속도가 비교적 빠름
rootZip = f'/content/drive/MyDrive/data/{zipName}.zip'
targetZip = f'/content/{zipName}.zip'
shutil.copyfile(rootZip, targetZip)

# zipfile.ZipFile로 압축 파일을 열고, 압축된 모든 파일을 targetPath로 이동(압축 해제)
with zipfile.ZipFile(targetZip, 'r') as zip_ref:
  zip_ref.extractall(targetPath)

## 데이터 Split

In [ ]:
''' Train:Val:Test = 6:2:2 분할'''
# rawdata를 분할. train:(val+test) = 6:4
train, temp = train_test_split(rawdata, test_size=0.4, random_state=1)

# rawdata를 분할. val:test = 5:5
val, test = train_test_split(temp, test_size=0.5, random_state=1)

## 데이터셋 클래스, Transform 생성

In [ ]:
''' DataSet 클래스 정의 '''
# =============================
# 목적 : DataFrame에 저장된 이미지 경로와 라벨 등을 불러오고, 전처리 후 (image, label) 리턴
# 매개변수(?)
#  - dataframe : 이미지 파일 이름/클래스 등이 포함된 변수
#  - rootDir : 이미지들이 저장된 경로
#  - transform : torchvision.transforms를 사용한 이미지 전처리 파이프라인(전처리 묶음? 정도로 이해하면 될 듯)
# =============================
class CustomDataset(Dataset):
  def __init__(self, dataframe, rootDir, transform=None):
    self.dataframe = dataframe
    self.rootDir = rootDir
    self.transform = transform

  # 데이터셋의 총 샘플 수 반환
  def __len__(self):
    return len(self.dataframe)

  # idx번째 데이터 리턴
  def __getitem__(self, idx):
    # DataFrame의 idx번째 행을 row에 저장
    row = self.dataframe.iloc[idx]
    # 이미지 경로 불러오기(row['image']는 파일명이므로, 상위 경로와 더해줌; 파일 이름이 '.JPG'로 된 경우도 있어서 통일)
    imgName = os.path.join(self.rootDir, row['image'].replace('.JPG', '.jpg'))

    # 이미지 파일을 열고, RGB로 변환
    image = Image.open(imgName).convert('RGB')

    # trasform이 정의되어 있다면, 전처리 적용
    if self.transform:
        image = self.transform(image)

    # 라벨 저장
    label = row['class']

    # (전처리된 이미지, 라벨) 리턴
    return image, label

In [ ]:
'''Transform 정의'''
# train, val, test 용도에 따라 전처리를 다르게 적용
if use_agumentation:
  transform = {
      'train': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299)),
          # 확률적으로 이미지를 좌우 반전 (데이터 증강)
          transforms.RandomHorizontalFlip(),
          # -15 ~ +15도 사이로 랜덤 회전 (데이터 증강)
          transforms.RandomRotation(15),
          # 밝기, 대비, 채도 랜덤 조절 (데이터 증강)
          transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
          # 10 % 비율로 좌우 또는 상화 랜덤 이동 (데이터 증강)
          transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
          # PIL 이미지를 Tensor 형식으로 변환 (0~255 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로 사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'val': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299)),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'test': transforms.Compose([ # val과 동일/ train과 달리 평가 용도기 때문에 val과 test에는 데이터 증강이 없음.
          transforms.Resize((299, 299)),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ])
  }
else:
  transform = {
      'train': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299)),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로 사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'val': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299)),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'test': transforms.Compose([ # val과 동일/ train과 달리 평가 용도기 때문에 val과 test에는 데이터 증강이 없음.
          transforms.Resize((299, 299)),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ])
  }

In [ ]:
# Dataset 불러오기
trainDataset = CustomDataset(train, '/content/data', transform=transform['train'])
valDataset = CustomDataset(val, '/content/data', transform=transform['val'])
testDataset = CustomDataset(test, '/content/data', transform=transform['test'])

# DataLoader 불러오기
trainLoader = DataLoader(trainDataset, batch_size=32, shuffle=False)
valLoader = DataLoader(valDataset, batch_size=32, shuffle=False)
testLoader = DataLoader(testDataset, batch_size=32, shuffle=False)

## 학습, 테스트 메소드 정의

In [ ]:
''' Inception v4 모델 생성 및 정의 '''

import torch
import torch.nn as nn
import timm  # Inception v4는 timm을 통해 불러옴

# 사용할 디바이스 설정 : GPU(cuda)가 가능하면 GPU, 아니면 CPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =============================
# 목적 : 모델을 불러오고 재정의
# =============================
def create_model():
    # 사전 학습된 Inception v4 모델 불러오기
    # ImageNet으로 학습된 가중치를 불러와서 학습 시간을 줄이고 성능 향상을 기대할 수 있음.
    model = timm.create_model('inception_v4', pretrained=True)

    # 기존의 fully connected layer는 제거하고, 새로운 분류기로 대체
    # 기존 fc 레이어의 feature 수 불러오기
    num_features = model.get_classifier().in_features  # Inception v4의 최종 FC는 classif에 있음

    # 새로운 fc 레이어 정의 (Sequential)
    model.head_drop = nn.Dropout(0.2*use_dropout)

    model.classif = nn.Sequential(
        # 기존 특성 수 >> 256 차원으로 축소
        nn.Linear(num_features, 256),
        # 활성화 함수
        nn.ReLU(),
        # 일부 뉴런을 50% 확률로 0으로 설정 >> 과적합 방지
        nn.Dropout(0.2*use_dropout),
        # 256 -> 2 차원으로 축소 (2개 클래스 분류를 위한 출력층)
        nn.Linear(256, 2),
    )

    # 모델을 지정한 디바이스로 이동
    return model.to(device)

In [ ]:
# 모델 불러오기
model = create_model()

print(model)

InceptionV4(
  (features): Sequential(
    (0): ConvNormAct(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNormAct2d(
        32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
    )
    (1): ConvNormAct(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
    )
    (2): ConvNormAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
    )
    (3): Mixed3a(
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (c

In [ ]:
''' 학습 모델 '''
# =============================
# 목적 : 모델 학습
# Early Stopping : val loss가 5번 이상 개선이 안될 경우, 종료
# =============================
def TrainModel(model, criterion, optimizer, scheduler, trainLoader, valLoader, numEpochs=20, patience=5):
    # Early Stopping 관련 변수
    # loss 기준값을 최댓값으로 설정
    best_val_loss = float('inf')
    # early stopping counter 0으로 설정
    counter = 0
    # best model 저장 경로 설정
    best_model_path = f"/content/drive/MyDrive/model/Inception v4 사전학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model.pth"

    # Backbone 동결 상태에서 fc layer만 학습
    for epoch in range(numEpochs):
        # Train
        # 모델을 학습 모드로 설정
        model.train()
        runningLoss = 0.0
        runningCorrects = 0

        # 훈련 데이터 반복 학습
        for inputs, labels in trainLoader:
            # 이미지와 라벨(클래스)
            inputs, labels = inputs.to(device), labels.to(device)

            # 기존 그래디언트 초기화
            optimizer.zero_grad()
            # 모델 forward pass
            outputs = model(inputs)
            # 손실함수 계산 (출력, 정답)
            loss = criterion(outputs, labels)
            # 역전파 학습
            loss.backward()
            # 파라미터 업데이트
            optimizer.step()

            # 예측값 계산
            _, preds = torch.max(outputs, 1)
            # 손실 합산
            runningLoss += loss.item() * inputs.size(0)
            # 정확도 합산
            runningCorrects += torch.sum(preds == labels.data)

        # 반복 횟수 단위로 Train 평균 손실 및 정확도 계산
        epochLoss = runningLoss / len(trainLoader.dataset)
        epochAcc = runningCorrects.double() / len(trainLoader.dataset)
        print(f"Epoch {epoch+1}/{numEpochs} - Train Loss: {epochLoss:.4f} Acc: {epochAcc:.4f}")

        # Validation
        # 모델을 평가 모드로 설정
        model.eval()
        valLoss = 0.0
        valCorrects = 0

        # 그래디언트 비활성화
        with torch.no_grad():
            for inputs, labels in valLoader:
                # 이미지와 라벨(클래스)
                inputs, labels = inputs.to(device), labels.to(device)
                # 모델 forward pass
                outputs = model(inputs)
                # 손실함수 계산 (출력, 정답)
                loss = criterion(outputs, labels)

                # 예측값 계산
                _, preds = torch.max(outputs, 1)
                # 손실 합산
                valLoss += loss.item() * inputs.size(0)
                # 정확도 합산
                valCorrects += torch.sum(preds == labels.data)

        # 반복 횟수 단위로 Validation 평균 손실 및 정확도 계산
        valLoss /= len(valLoader.dataset)
        val_acc = valCorrects.double() / len(valLoader.dataset)
        print(f"Validation Loss: {valLoss:.4f} Acc: {val_acc:.4f}")

        # Scheduler: Validation Loss 기반으로 학습률 조정
        scheduler.step(valLoss)

        # Early Stopping
        # 개선이 되면
        if valLoss < best_val_loss:
            best_val_loss = valLoss
            counter = 0
            torch.save(model, best_model_path)
            print(f"Renewal best model: {best_val_loss:.4f}")
        # 개선이 안되면
        else:
            counter += 1
            print(f"No improvement in {counter}/{patience} epochs")
            if counter >= patience:
                break

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
import numpy as np
import torch.nn.functional as F

def TestModel(model, testLoader):
    all_preds = []
    all_probs = []  # AUC 계산용 softmax 확률
    all_labels = []

    testCorrects = 0
    testLoss = 0.0

    criterion = nn.CrossEntropyLoss()
    model.eval()

    with torch.no_grad():
        for inputs, labels in testLoader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 예측 클래스
            _, preds = torch.max(outputs, 1)
            # softmax 확률 (클래스 1의 확률만)
            probs = F.softmax(outputs, dim=1)[:, 1]

            testLoss += loss.item() * inputs.size(0)
            testCorrects += torch.sum(preds == labels.data)

            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 평균 계산
    testLoss /= len(testLoader.dataset)
    testAcc = testCorrects.double() / len(testLoader.dataset)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    auc = roc_auc_score(all_labels, all_probs)

    print(f"Test Accuracy  : {testAcc:.4f}")
    print(f"Test Loss      : {testLoss:.4f}")
    print(f"ROC-AUC        : {auc:.4f}")
    print(f"Precision      : {precision:.4f}")
    print(f"Recall         : {recall:.4f}")
    print(f"F1-macro       : {f1:.4f}")

    return testAcc.item(), testLoss, auc, precision, recall, f1

## 모델 학습 및 테스트

In [ ]:
for i in range(0,2):
  # 모델 불러오기
  model = create_model()

  # CrossEntropyLoss로 손실 함수 설정
  criterion = nn.CrossEntropyLoss()
  # Adam으로 옵티마이저로 사용 (filter를 통해 require_grad = True, 동결되지 않은 것만 업데이트)
  # weight_decay = 4e-5 : L2 정규화 적용으로 과적합 방지 / 논문 l2 정규화 값 0.00004
  optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=4e-5*use_l2)

  # 학습률 스케줄러 설정
  # 검증 손실이 줄어들지 않으면 learning rate 줄임 (수렴 및 안정화)
  # min 모드 : 손실이 최소화되지 않으면 작동 / factor : 학습률을 x배 줄임 / patience : x번 학습동안 개선되지 않으면 발동 / verbose : 메시지 출력
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

  # Train 모델 실행
  TrainModel(model, criterion, optimizer, scheduler, trainLoader, valLoader, numEpochs=20, patience=5)

  os.rename(f"/content/drive/MyDrive/model/Inception v4 사전학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model.pth", f"/content/drive/MyDrive/model/Inception v4 사전학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model_{i+1}.pth")

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20 - Train Loss: 2.4328 Acc: 0.6667
Validation Loss: 0.2259 Acc: 0.9612
Renewal best model: 0.2259
Epoch 2/20 - Train Loss: 0.1750 Acc: 0.9696
Validation Loss: 0.1285 Acc: 0.9729
Renewal best model: 0.1285
Epoch 3/20 - Train Loss: 0.0423 Acc: 0.9897
Validation Loss: 0.0500 Acc: 0.9845
Renewal best model: 0.0500
Epoch 4/20 - Train Loss: 0.0258 Acc: 0.9948
Validation Loss: 0.0603 Acc: 0.9845
No improvement in 1/5 epochs
Epoch 5/20 - Train Loss: 0.0132 Acc: 0.9974
Validation Loss: 0.0622 Acc: 0.9845
No improvement in 2/5 epochs
Epoch 6/20 - Train Loss: 0.0288 Acc: 0.9929
Validation Loss: 0.0592 Acc: 0.9845
No improvement in 3/5 epochs
Epoch 7/20 - Train Loss: 0.0204 Acc: 0.9935
Validation Loss: 0.0492 Acc: 0.9864
Renewal best model: 0.0492
Epoch 8/20 - Train Loss: 0.0056 Acc: 0.9994
Validation Loss: 0.0487 Acc: 0.9884
Renewal best model: 0.0487
Epoch 9/20 - Train Loss: 0.0108 Acc: 0.9968
Validation Loss: 0.0868 Acc: 0.9729
No improvement in 1/5 epochs
Epoch 10/20 - Train Loss: 0.0

In [ ]:
print(f"{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model.pth")
for i in range(2):
  best_model_path = f"/content/drive/MyDrive/model/Inception v4 사전학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model_{i+1}.pth"
  model = torch.load(best_model_path, weights_only=False)
  model.to(device)

  # 테스트
  result = TestModel(model, testLoader)
  print()

1111_best_model.pth
Test Accuracy  : 0.9864
Test Loss      : 0.0536
ROC-AUC        : 0.9978
Precision      : 0.9865
Recall         : 0.9864
F1-macro       : 0.9864

Test Accuracy  : 0.9903
Test Loss      : 0.0528
ROC-AUC        : 0.9981
Precision      : 0.9903
Recall         : 0.9903
F1-macro       : 0.9903

